# 作用域

作用域是指变量名的有效范围，是为了解决开发过程中变量名冲突发展起来的概念。

**Ruby** 语言是赋值即变量定义的语言，他没有与 **JavaScript** 语言的 `var x` 和 **Java** 的 `String str` 相当的变量声明。执行函数中的语句就定义了局部变量。

In [18]:
x = 1
def change_x
  x = 2
end

change_x
x

1

In [28]:
%%file change_x.py

x = 1
def change_x():
  x = 2

change_x()
print("x = {0}".format(x))

Writing change_x.py


In [29]:
puts `python change_x.py`

x = 1



如你所料，`x` 的值并没有真正被修改。这和 **Python** 表现是一致的，但是其深层的原理却有比较大的差别。

在 **Python** 中，`change_x` 是一个闭包，它可以看到外部的 `x`，之所以没有修改成功，是因为 `change_x` 重新定义了本地变量 `x`，并把 `2` 赋值给了 `x`。

在 **Ruby** 中，`change_x` 则是打开了一个全新的作用域，它是无法访问到外部的变量 `x` 的。

In [30]:
%%file check_x.py

x = 1
def check_x():
  print("I can see x = {0} in check_x.".format(x))

check_x()

Writing check_x.py


In [31]:
puts `python check_x.py`

I can see x = 1 in check_x.



## Scope Gate

每个 **Ruby** 作用域包含一组绑定，并且不同的作用域之间被 **Scope Gate** 分割开来。**Scope Gate** 的作用是关闭前一个作用域，同时打开一个新的作用域。**Ruby** 中使用 `class`，`module` 或 `def` 关键字来隔离作用域，也就是充当 **Scope Gate**。

In [5]:
main0 = 'main0'

module MyModule
  module1 = 'module1'
  puts "enter MyModule locals: #{local_variables}"
  
  class MyClass
    class2 = 'class2'
    puts "enter MyClass locals: #{local_variables}"
    
    def my_method
      method_3 = 'method3'
      puts "my_method locals: #{local_variables}"
    end
    
    puts "exit MyClass locals: #{local_variables}"
  end
  
  puts "exit MyModule locals: #{local_variables}"
end
    
obj = MyModule::MyClass.new
obj.my_method
obj.my_method
  
puts "main locals: #{local_variables}"

enter MyModule locals: [:module1]
enter MyClass locals: [:class2]
exit MyClass locals: [:class2]
exit MyModule locals: [:module1]
my_method locals: [:method_3]
my_method locals: [:method_3]
main locals: [:_i4, :_4, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :_, :__, :___, :_i1, :_1, :main0, :obj, :_oh, :_ih, :title]


容易看出程序打开了五个独立的作用域

- 顶级作用域（`main0` 所在的作用域）
- 进入 `MyModule` 时创建的作用域
- 进入 `MyClass` 时创建的作用域
- 第一次调用 `my_method()` 方法时创建的一个作用域
- 第二次调用 `my_method()` 方法时创建的一个作用域

如何知道每次调用同一方法时会重新创建新的作用域而不是重用已有的，我们通过一个程序来验证。

In [22]:
class MyClass
  def my_method
    v = rand(1..100)
    puts "my_method locals: #{local_variables}"
    binding
  end 
end

obj = MyClass.new
binding1 = obj.my_method
binding2 = obj.my_method

puts "v in binding1 is #{binding1.local_variable_get(:v)}"
puts "v in binding2 is #{binding2.local_variable_get(:v)}"

binding1.local_variable_set(:v2, 1)

puts "binding1 variables is #{binding1.local_variables}"
puts "binding2 variables is #{binding2.local_variables}"

my_method locals: [:v]
my_method locals: [:v]
v in binding1 is 58
v in binding2 is 27
binding1 variables is [:v2, :v]
binding2 variables is [:v]


`class` / `module` 与 `def` 之间还有一点微妙的差别，在类和模块定义中的代码会被立即执行，相反，方法定义中的代码只有在方法被调用的时候才会被执行。

## 顶级作用域（Top Level Scope）

当开始运行 **Ruby** 程序时，**Ruby** 虚拟机会创建一个名为 `main` 的对象作为当前对象，这个对象被称为顶级上下文，这个名字的由来是因为这时处在调用栈的顶层，这时要么还没调用任何方法，要么调用的所有方法都已经返回了。

In [24]:
puts self
puts self.class
puts local_variables

main
Object
[:_i20, :_20, :_i19, :_19, :_i18, :_18, :_i17, :_17, :_i16, :_16, :_i15, :_15, :_i14, :_14, :_i13, :_13, :_i12, :_12, :binding1, :binding2, :_i11, :_11, :_i10, :_10, :_i9, :_9, :_i8, :_8, :_i7, :_7, :_i6, :_6, :_i5, :_5, :_i4, :_4, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :_, :__, :___, :_i1, :_1, :main0, :obj, :_oh, :_ih, :title]


In [27]:
self.binding.local_variables

NoMethodError: private method `binding' called for main:Object



变量的作用域及生命周期的关系。


静态语义（保持一致）
语法作用域（讨论代码的组织结构上的抽象，讨论的是“圈地”的问题 【形式上的规范】）   静态作用域
变量作用域（变量的可见性，完成对信息的隐藏，也就是处理“割据”问题 【实际的占有】）       动态作用域


在冯·诺依曼计算机体系结构的内存中，变量的属性可以视为一个六元组：（名字，地址，值，类型，生命期，作用域）。地址属性具有明显的冯·诺依曼体系结构 的色彩，代表变量所关联的存储器地址。类型规定了变量的取值范围和可能的操作。生命期表示变量与某个存储区地址绑定的过程。根据生命期的不同，变量可以被 分为四类：静态、栈动态、显式堆动态和隐式堆动态。作用域表征变量在语句中的可见范围，分为词法作用域和动态作用域两种。

闭包的数据结构定义，包含一个函数定义 f 和它定义时所在的环境(struct Closure (f env))。

闭包是一种特殊的数据结构，它由两个元素组成：函数的定义和当前的环境。所以我们对 (lambda (x) e) 这样一个函数的解释就是这样：

      [`(lambda (,x) ,e)
       (Closure exp env)]

注意这里的 exp 就是 `(lambda (,x) ,e) 自己。我们只是把它包装了一下，把它与当前的环境一起放到一个数据结构(闭包)里，并不进行任何复杂的运算。这里我们的闭包用的是一个 Racket 的 struct 结构，也就是一个记录类型(record)。你也可以用其它形式来表示闭包，比如有些解释器教程提倡用函数来表示闭包。其实用什么形式都无所谓，只要能存储 exp 和 env 的值。我比较喜欢使用 struct，因为它的界面简单清晰。


In [2]:
title

"ruby /usr/local/rvm/gems/ruby-2.4.1/bin/iruby kernel /Users/james/Library/Jupyter/runtime/kernel-947f3fa0-aac0-4879-b7b1-fcb884ee0290.json"

在函数中，`Ruby` 从命名空间中寻找变量的顺序如下：

- `local function scope`
- `enclosing scope`
- `global scope`
- `builtin scope`

例子：

# local 作用域

In [1]:
def foo(a,b):
    c = 1
    d = a + b + c

这里所有的变量都在 `local` 作用域。

## global 作用域

In [2]:
c = 1
def foo(a,b):
    d = a + b + c

这里的 `c` 就在 `global` 作用域。

## global 关键词

使用 `global` 关键词可以在 `local` 作用域中修改 `global` 作用域的值。

In [3]:
c = 1
def foo():
    global c
    c = 2
    
print c
foo()
print c

1
2


其作用是将 `c` 指向 `global` 中的 `c`。

如果不加关键词，那么 `local` 作用域的 `c` 不会影响 `global` 作用域中的值：

In [4]:
c = 1
def foo():
    c = 2
    
print c
foo()
print c

1
1


## built-in 作用域

In [5]:
def list_length(a):
    return len(a)

a = [1,2,3]
print list_length(a)

3


这里函数 `len` 就是在 `built-in` 作用域中：

In [6]:
import __builtin__

__builtin__.len

<function len>

## class 中的作用域

Global | MyClass
---|---
`var = 0` <br> `MyClass` <br> `access_class` | `var = 1`<br>`access_class` 

In [7]:
# global
var = 0

class MyClass(object):
    # class variable
    var = 1
    
    def access_class_c(self):
        print 'class var:', self.var
    
    def write_class_c(self):
        MyClass.var = 2
        print 'class var:', self.var
        
    def access_global_c(self):
        print 'global var:', var
    
    def write_instance_c(self):
        self.var = 3
        print 'instance var:', self.var

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class`] <br> `obj` | `var = 1`<br>`access_class`  |

In [8]:
obj = MyClass()

查询 `self.var` 时，由于 `obj` 不存在 `var`，所以跳到 MyClass 中：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 1`<br>`access_class`  |

In [9]:
obj.access_class_c()

class var: 1


查询 `var` 直接跳到 `global` 作用域：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 1`<br>`access_class`  |

In [10]:
obj.access_global_c()

global var: 0


修改类中的 `MyClass.var`：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 2`<br>`access_class`  |

In [11]:
obj.write_class_c()

class var: 2


修改实例中的 `var` 时，会直接在 `obj` 域中创建一个：

Global | MyClass | obj
---|---|----
`var = 0` <br> `MyClass` <br> [`access_class` <br> `self`] <br> `obj` | `var = 2`<br>`access_class`  | `var = 3`

In [12]:
obj.write_instance_c()

instance var: 3


In [13]:
MyClass.var

2

`MyClass` 中的 `var` 并没有改变。

## 词法作用域

对于嵌套函数：

In [14]:
def outer():
    a = 1
    def inner():
        print "a =", a
    inner()
    
outer()

a = 1


如果里面的函数没有找到变量，那么会向外一层寻找变量，如果再找不到，则到 `global` 作用域。

返回的是函数的情况：

In [15]:
def outer():
    a = 1
    def inner():
        return a
    return inner
    
func = outer()

print 'a (1):', func()

a (1): 1


func() 函数中调用的 `a` 要从它定义的地方开始寻找，而不是在 `func` 所在的作用域寻找。